This is the notebook for the course capstone project - Week 3.

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

# get the data from Wikipage 

link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

## use soup to get the informaion 

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

soup = soup.find_all('table')

## Create array to hold the data we extract

In [ ]:
postal_codes = []
boroughs = []
neighborhoods = []

table = soup[0]
rows = table.find_all('tr')

for row in rows:
    cells = row.find_all('td')


    if len(cells) > 1:
        
        postal_code = cells[0]
        postal_codes.append(postal_code.text.strip())

        borough = cells[1]
        boroughs.append(borough.text.strip())

        neighborhood = cells[2]
        neighborhoods.append(neighborhood.text.strip())

## convert to DataFrame

In [ ]:
df = pd.DataFrame(postal_codes,columns=['PostalCode'])

df['Borough'] = boroughs

df['Neighborhood'] = neighborhoods

df.head()

## drop the not assigned columns 

In [ ]:
df[df['Borough'] == 'Not assigned'] = None

df.dropna(axis=0,inplace=True)

## check if there are postal codes with multiple neighborhoods  

when getting the data from the web, if one postal code has multiple neighborhoods, they are combined together with ','

In [ ]:
len(df['PostalCode'].unique()) == len(df['PostalCode'])

## print the number of rows 

In [ ]:
print('Number of rows:',df.shape[0])

In [ ]:
## insert latitude and longitude 

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

In [ ]:
postal_code = df['PostalCode'].iloc[1]
postal_code

In [ ]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

In [ ]:
df = df.merge(df_geo, left_on='PostalCode', right_on='Postal Code')
df.drop(['Postal Code'],axis=1,inplace=True)
df.head()

## create a map

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
import folium # map rendering library

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## find all neighborhoods with Toronto in them 

In [ ]:
df_Borough_toronto = df[df['Borough'].str.contains("Toronto")]

df_Borough_toronto.head()

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Borough_toronto['Latitude'], df_Borough_toronto['Longitude'], df_Borough_toronto['Borough'], df_Borough_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto